# This notebook builds a model using Bayesian optimization and mel-spectrograms as input feature to the model.

In [7]:
import time

import keras_tuner
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.models import Sequential

from methods_audio import data_handling

# 1. Get data (file names)

In [2]:
data = data_handling.get_data()

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



# 2. Read data (transforming file names into waves) <br>
Additionally, the mean is removed and the data is normalized. 

In [3]:
data = data.map(data_handling.read_in_data)

# 3. Get input for model training 

In [4]:
samples, labels = data_handling.extract_samples_labels(data)

2023-05-14 09:59:28.185802: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


# 4. Split data into train and validation sets

In [5]:
validation_set_size = 0.20
x_train, x_valid, y_train, y_valid = train_test_split(samples, labels, test_size=validation_set_size, random_state=123)

# 5. Transform data to db mel-spectrograms

In [6]:
type_transformation = "db_mel_spectrogram"
x_train = data_handling.transform_data(x_train, type_transformation)
x_valid = data_handling.transform_data(x_valid, type_transformation)

## 6. Build model with hyperparameter tunning 
https://keras.io/guides/keras_tuner/getting_started/ <br>
https://www.youtube.com/watch?v=6Nf1x7qThR8&ab_channel=GregHogg 

In [8]:
def build_model(hp):
    input = (624, 128, 1)
    model = keras.Sequential()

    # Add input layer
    # matching samples.shape
    model.add(
        Conv2D(
            filters=hp.Int("conv_filters_0", min_value=8, max_value=128, step=16),
            activation=hp.Choice("conv_activation_0", ["relu", "tanh"]),
            kernel_size=(3, 3),
            input_shape=input,
        )
    )
    model.add(MaxPool2D(pool_size=(2, 2)))

    # Tune the number of Conv layers
    for i in range(hp.Int("num_conv_layers", 1, 4)):
        model.add(
            Sequential(
                [
                    layers.Conv2D(
                        filters=hp.Int(f"conv_filters_{i}", min_value=8, max_value=128, step=16),
                        activation=hp.Choice(f"conv_activation_{i}", ["relu", "tanh"]),
                        kernel_size=(4, 3),
                    ),
                    layers.MaxPool2D(pool_size=(2, 2)),
                ]
            )
        )

    model.add(layers.Flatten())

    # Tune the number of Dense layers and Tune whether to use dropout layer
    for i in range(hp.Int("num_dense_layers", 1, 6)):
        model.add(
            Sequential(
                [
                    layers.Dense(
                        # Tune number of units separately.
                        units=hp.Int(f"dense_units_{i}", min_value=50, max_value=600, step=50),
                        activation=hp.Choice(f"dense_activation_{i}", ["relu", "tanh"]),
                    ),
                    layers.Dropout(rate=hp.Float(f"dense_dropout_{i}", min_value=0, max_value=1)),
                ]
            )
        )

    model.add(
        layers.Dense(
            units=1,  # because we have 2 classes
            activation=hp.Choice("activatio_last_layer", ["softmax", "sigmoid"]),
        )
    )

    # Define the optimizer learning rate as a hyperparameter.
    # sampling="log", the step is multiplied between samples.
    lr = hp.Float("learning_rate", min_value=1e-4, max_value=1e-1, sampling="log")
    model.compile(
        optimizer=keras.optimizers.SGD(learning_rate=lr),
        loss="BinaryCrossentropy",
        metrics=["accuracy"],
    )

    return model

In [9]:
build_model(keras_tuner.HyperParameters())

### Initialize tuner by specifying different arguments 

In [10]:
tuner = keras_tuner.BayesianOptimization(
    hypermodel=build_model,
    objective="val_accuracy",  # we want maximize accuracy
    max_trials=50,
    overwrite=True,
    directory="param_optimization",
    project_name="db_mel_spectrogram",
)

In [11]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5)
# patience refers to number of epochs: if the val loss is not improving fter 5 ephocs, we stop it.

### During the search, the model is called with different hyperparameters 

In [23]:
tuner.search_space_summary()
# Default search space size: number of hyper parameters that we are tunning

Search space summary
Default search space size: 9
conv_filters_0 (Int)
{'default': None, 'conditions': [], 'min_value': 8, 'max_value': 128, 'step': 16, 'sampling': 'linear'}
conv_activation_0 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
num_conv_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 4, 'step': 1, 'sampling': 'linear'}
num_dense_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 6, 'step': 1, 'sampling': 'linear'}
dense_units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 50, 'max_value': 600, 'step': 50, 'sampling': 'linear'}
dense_activation_0 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
dense_dropout_0 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 1.0, 'step': None, 'sampling': 'linear'}
activatio_last_layer (Choice)
{'default': 'softmax', 'conditions': [], 'values': ['softmax', 'sigmo

In [ ]:
epochs = 50
start_time = time.time()

tuner.search(
    np.stack(x_train),
    np.stack(y_train),
    epochs=epochs,
    validation_data=(np.stack(x_valid), np.stack(y_valid)),
    callbacks=[stop_early],
)  # similar to fit

end_time = time.time()

In [ ]:
elapsed_time = end_time - start_time
print(f"The search took {elapsed_time:.2f} seconds to finish.")

In [ ]:
tuner.results_summary()

### After all of that we don't have a model yet but rather a set of hyper parameters. Let's query the results and create a model:

In [ ]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(best_hps.values)

In [ ]:
learning_rate = best_hps.get("learning_rate")
print(learning_rate)

In [ ]:
model = tuner.hypermodel.build(best_hps)

In [ ]:
location = "data/models/db_mel_spectrogram"
model.save(location)